# Exercise 12 – Feature engineering 

**General Instructions:**

- Collaborations between students during problem-solving phase on a discussion basis is OK
- However: individual code programming and submissions per student are required
- Code sharing is strictly prohibited
- We will run checks for shared code, general plagiarism and AI-generated solutions
- Any fraud attempt will lead to an auto fail of the entire course
- Do not use any additional packages except for those provided in the task templates
- Please use Julia Version 1.10.x to ensure compatibility
- Please only write between the `#--- YOUR CODE STARTS HERE ---#` and `#--- YOUR CODE ENDS HERE ---#` comments
- Please do not delete, add any cells or overwrite cells other than the solution cells (**Tip:** If you use a jupyerhub IDE, you should not be able to add or delete cells and write in the non-solution cells by default)

In [ ]:
using Pkg
Pkg.activate(@__DIR__)

In [ ]:
using LinearAlgebra
using Plots
using HDF5
using MAT
using Flux
using Serialization
using Random
using Statistics

## Task 1 – Extended Dynamic Mode Decomposition using a dictionary
Learn a linear dynamical system from trajectory data via the Dynamic Mode Decomposition. As the example, we will use the Duffing oscillator with a two-dimensional state $x\in\mathbb{R}^2$. The function for the right-hand side is given below

In [ ]:
function rhs(x)
    ɑ, β, δ = -1, 1, 0.1
    xdot = [x[2], - δ * x[2] - ɑ * x[1] - β * x[1]^3]
    return xdot
end

a) Create training data $X$ and $X’$. To this end, draw $N=1000$ initial conditions randomly and uniformly from the rectangle $[-2,2] \times [-2,2]$ and perform one time step using the explicit Euler scheme with a time step of $h = 0.1$. 

In [ ]:
X = nothing
X_prime = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
@assert size(X) == (2, 1000)
@assert size(X_prime) == (2, 1000)


b) Use DMD in its standard form to learn a linear system with $A\in\mathbb{R}^{2\times 2}$. Simulate the dynamics over $m=100$ time steps using the initial condition $x_0=[1.0, -1.0]$. Report on the RMSE between the created trajectory and a true trajectory of the system (same initial condition, explicit Euler integration).

Note: To avoid confusion regarding the RMSE on multiple dimensions, take this one:
$ \text{RMSE} = \sqrt{ \frac{1}{T} \sum_{t = 1}^{T} \sum_{i = 1}^{d} (X_{t, i} - Y_{t, i})^2 } $

In [ ]:
x0 = [1.0, -1.0]

A = nothing
dmd_trajectory = nothing
euler_trajectory = nothing
RMSE = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#


In [ ]:
@assert size(A) == (2, 2)
@assert size(dmd_trajectory) == (2, 100)
@assert size(euler_trajectory) == (2, 100)
@assert isa(RMSE, Number)


c) Implement a dictionary of radial basis functions (RBFs), i.e.,  
$$ \varphi_c(x) = \exp(- \gamma \| x - c \|_2^2 ),$$ 
$$ z = \Psi(x) = [\varphi_{c_1}(x), \ldots, \varphi_{c_r}(x)],$$ 
where the $c_i\in\mathbb{R}^2$ are the centers of the individual RBFs and $\gamma>0$ is a hyperparameter determining the width of the RBF 

In [ ]:
# Function to compute the lifted state using a dictionary of RBFs
function rbf_dictionary(x, centers, gamma=1.5)
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
end

In [ ]:
@assert isa(rbf_dictionary, Function)
@assert length(rbf_dictionary([1, 1], [[2, 2], [3, 3]])) == 2
@assert length(rbf_dictionary([1, 1], [[2, 2], [3, 3], [4, 4]])) == 3


d) Use the extended version of DMD (aka EDMD), where you find a linear system for the lifted state $\Psi(x)=z\in\mathbb{R}^r$ instead of $x\in\mathbb{R}^2$ (use the trajectory from a) ). To this end, introduce $r=100$ centers on an equidistant $10 \times 10$ grid in the area $[-2, 2] \times [-2, 2]$, with $\gamma = 1$. Train the corresponding matrix $A\in\mathbb{R}^{100 \times 100}$. 

In [ ]:
centers = nothing
Z = nothing
Z_prime = nothing
A_edmd = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
@assert size(A_edmd) == (100, 100)
@assert length(centers) == 100
@assert size(Z) == (100, 1000)
@assert size(Z_prime) == (100, 1000)


e) Implement a projection operation that maps from $z$ to $x$, such that you can reconstruct the original state $x$ from predictions of the lifted state $z$. This step can be realized by a linear mapping $x = P z$, where the projection matrix $P\in\mathbb{R}^{2\times r}$ can be trained using linear regression. 

In [ ]:
P = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
@assert size(P) == (2, 100)


f) Repeat the experiment from b), but now compare the original trajectory to the one obtained using EDMD and the consecutive projection step to recover $x$. 

```Hint: Don't forget to transform the initial condition accordingly```

In [ ]:
edmd_trajectory_lifted = nothing
edmd_trajectory_projected = nothing
error_edmd = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
@assert size(edmd_trajectory_lifted) == (100, 100)
@assert size(edmd_trajectory_projected) == (2, 100)
@assert isa(error_edmd, Number)


## Task 2 – Comparing SVD and Autoencoders
In this exercise, we want to compare the capabilities of the Singular Value Decomposition (SVD) and an autoencoder in terms of data compression of high-dimensional trajectory data. The following data set $X\in\mathbb{R}^{101 \times 6001}$ has been created by simulating the Burgers PDE (https://en.wikipedia.org/wiki/Burgers%27_equation) on an $n = 101$-dimensional spatial grid for $N=6000$ time steps. 

a) Use the SVD to compress the $n$-dimensional state into a much lower dimension $r\ll n$. Select the smallest value for $r$ for which the reconstruction error between the original data $x$ and the reconstructed data $\tilde{x}$ is less than 0.1%, i.e. 
$$ \frac{ \sum_{i=1}^N \|x_i – \tilde{x}_i \|_2^2 }{ \sum_{i=1}^N \|x_i \|_2^2 } < 0.001. $$ 

In [ ]:
#Data loading
file = matopen("burgers.mat")
X = read(file, "u")
close(file)

In [ ]:
function loss_function(original, reconstructed)
    #--- YOUR CODE STARTS HERE ---#
    
    #--- YOUR CODE ENDS HERE ---#
end

In [ ]:
@assert isa(loss_function, Function)


In [ ]:
k = nothing
X_reconstructed_k = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

In [ ]:
@assert isa(k, Number)
@assert size(X_reconstructed_k) == (101, 6001)
@assert loss_function(X, X_reconstructed_k) < 0.001


```
#Random SVD sample comparison
random_index = rand(1:size(X,2))
x = range(0.1,10.1, length = 101)
println("Original Image")
Plots.plot(x,X[:, random_index])
println("Reconstructed Image")
display(Plots.plot!(x,X_reconstructed_k[:, random_index]))
```

b) Now compare this to an autoencoder architecture of your choice. The only constraints that should be respected are: 
- Use only fully connected feed-forward layers (i.e., no convolutions) 
- the bottleneck layer connecting the encoder and the decoder (i.e., the smallest layer whose latent state is the compressed state) has to have dimension at most $r=5$. 
This task is fulfilled if your trained architecture can satisfy the error criterion from a) 

In [ ]:
#define your autoencoder
autoencoder = nothing

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#


In [ ]:
@assert isa(autoencoder, Flux.Chain)

**!!!The training cell below should be commented before you turn in your assignment!!!**

In [ ]:
# Train your autoencoder AND make it a comment before you turn in the assignment!!!!

#--- YOUR CODE STARTS HERE ---#

#--- YOUR CODE ENDS HERE ---#

**!!!The following cell stores your autoencoder, this file must be turned in in addition to your notebook!!!**

In [ ]:
#Make sure that the model is not overwritten during autograding
if @isdefined autoencoder
    #Save your model as file
    @assert isa(autoencoder, Flux.Chain)
    serialize("autoencoder", autoencoder)
end

In [ ]:
student_model = deserialize("autoencoder")
@assert isa(student_model, Flux.Chain)

@assert loss_function(X, autoencoder(X)) < 0.001



```
#Random autoencoder sample comparison
x = range(0.1,10.1, length = 101)
random_index = rand(1:size(X,2))
original = X[:,random_index]
reconstruction = autoencoder(original)
Plots.plot(x,original)
display(Plots.plot!(x,reconstruction))
```